In [6]:
!pip install --upgrade git+https://github.com/flairNLP/flair.git

  Cloning https://github.com/flairNLP/flair.git to /tmp/pip-req-build-jglogkum
  Running command git clone -q https://github.com/flairNLP/flair.git /tmp/pip-req-build-jglogkum
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 645kB 14.6MB/s 
     |████████████████████████████████| 256kB 21.8MB/s 
     |████████████████████████████████| 798kB 53.4MB/s 
     |████████████████████████████████| 983kB 43.0MB/s 
     |████████████████████████████████| 1.1MB 45.4MB/s 
     |████████████████████████████████| 890kB 47.3MB/s 
     |████████████████████████████████| 3.8MB 43.8MB/s 
  Created wheel for flair: filename=flair-0.4.5-cp36-none-any.whl size=148505 sha256=bb3d7605d349e1c56e20c26b9e3171568b7dbb697782c90ebcc826d16f37c25a
  Stored in directory: /tmp/pip-ephem-wheel-cache-a7kh_mgq/wheels/84/82/73/d2b3b59b7be74ea05f2c6d64132efe27df52daffb47d1dc7bb
Successfully built flair
  Cre

In [0]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings, DocumentRNNEmbeddings, BertEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
from flair.datasets import CSVClassificationCorpus
from flair.data import Corpus
import pandas as pd

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data_folder = "/content/drive/My Drive/Capstone/data/"

### First Stage (Train on benchmark dataset)

In [0]:
benchmark = pd.read_csv(data_folder + "combined_benchmark.csv")

In [0]:
benchmark = benchmark[['label', 'text']]
benchmark.head()

,label,text
0,0,Why not subscribe to the magazine ?
1,-1,"Tornio Works employs 2,300 of whom more than 1..."
2,1,"The move is aimed at boosting sales , cost-eff..."
3,0,"As a result of the merger , the largest profes..."
4,-1,18 March 2010 A leakage in the gypsum pond was...


#### Create train, dev and test set

In [0]:
benchmark = benchmark.sample(frac=1)
benchmark.iloc[0:int(len(benchmark)*0.8)].to_csv(data_folder + 'train.csv', sep='\t', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.8):int(len(benchmark)*0.9)].to_csv(data_folder + 'test.csv', sep='\t', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.9):].to_csv(data_folder + 'dev.csv', sep='\t', index = False, header = False)

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn

In [0]:
kaggle_X_train, kaggle_X_te, kaggle_Y_train, kaggle_Y_te = train_test_split(benchmark['text'],benchmark['label'], test_size = 0.2, random_state = 0, stratify = benchmark['label'])
kaggle_X_dev, kaggle_X_test, kaggle_Y_dev, kaggle_Y_test = train_test_split(kaggle_X_te,kaggle_Y_te, test_size = 0.5, random_state = 0)

In [0]:
benchmark_train_df =  pd.DataFrame({ 'label': kaggle_Y_train, 'text': kaggle_X_train   })
benchmark_dev_df = pd.DataFrame({ 'label': kaggle_Y_dev , 'text': kaggle_X_dev  })
benchmark_test_df = pd.DataFrame({ 'label': kaggle_Y_test, 'text': kaggle_X_test   })

In [0]:
benchmark_train_df.to_csv( data_folder + "train.csv", index = False, header = False)
benchmark_dev_df.to_csv( data_folder + "dev.csv", index = False, header = False)
benchmark_test_df.to_csv( data_folder + "test.csv", index = False, header = False)

#### Build corpus

In [0]:
# corpus = NLPTaskDataFetcher.load_classification_corpus(Path(data_folder), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
column_name_map = {1: "text", 0: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=False, #no header in kaggle data
                                         delimiter='\t',    # comma separated rows
)

2020-05-22 00:27:43,337 Reading data from /content/drive/My Drive/Capstone/data
2020-05-22 00:27:43,340 Train: /content/drive/My Drive/Capstone/data/train.csv
2020-05-22 00:27:43,345 Dev: /content/drive/My Drive/Capstone/data/dev.csv
2020-05-22 00:27:43,347 Test: /content/drive/My Drive/Capstone/data/test.csv


#### Create word embeddings

In [0]:
word_embeddings = [BertEmbeddings(), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (Use 'TransformerWordEmbeddings' for all transformer-based word embeddings) -- Deprecated since version 0.4.5.
  """Entry point for launching an IPython kernel.


#### First Stage Fine-tuning

In [0]:
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)
trainer = ModelTrainer(classifier, corpus)
trainer.train(data_folder, max_epochs=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  """Entry point for launching an IPython kernel.


2020-05-22 00:28:38,684 Computing label dictionary. Progress:


100%|██████████| 1314/1314 [00:01<00:00, 805.37it/s]

2020-05-22 00:28:40,610 [b'0', b'1', b'-1']
2020-05-22 00:28:40,623 ----------------------------------------------------------------------------------------------------
2020-05-22 00:28:40,627 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=Tru

2020-05-22 00:28:42,774 epoch 1 - iter 3/37 - loss 1.47190928 - samples/sec: 53.79
2020-05-22 00:28:52,919 epoch 1 - iter 6/37 - loss 1.48636297 - samples/sec: 59.70
2020-05-22 00:29:03,053 epoch 1 - iter 9/37 - loss 1.42424431 - samples/sec: 62.67
2020-05-22 00:29:13,600 epoch 1 - iter 12/37 - loss 1.33435683 - samples/sec: 64.34
2020-05-22 00:29:23,750 epoch 1 - iter 15/37 - loss 1.26441330 - samples/sec: 63.38
2020-05-22 00:29:33,755 epoch 1 - iter 18/37 - loss 1.22004982 - samples/sec: 65.24
2020-05-22 00:29:44,029 epoch 1 - iter 21/37 - loss 1.21731525 - samples/sec: 61.67
2020-05-22 00:29:53,974 epoch 1 - iter 24/37 - loss 1.23220348 - samples/sec: 67.34
2020-05-22 00:30:04,015 epoch 1 - iter 27/37 - loss 1.20821805 - samples/sec: 67.07
2020-05-22 00:30:14,026 epoch 1 - iter 30/37 - loss 1.17964252 - samples/sec: 64.34
2020-05-22 00:30:24,257 epoch 1 - iter 33/37 - loss 1.15531065 - samples/sec: 61.37
2020-05-22 00:30:34,405 epoch 1 - iter 36/37 - loss 1.15251064 - samples/sec: 6

{'dev_loss_history': [1.2375831604003906,
  0.9462193846702576,
  0.9772723317146301,
  0.8877629637718201,
  1.028766393661499,
  0.7877547144889832,
  1.1813462972640991,
  0.9026913642883301,
  0.7203583717346191,
  0.8858896493911743],
 'dev_score_history': [0.5918367346938775,
  0.6825396825396826,
  0.6507936507936508,
  0.7505668934240363,
  0.6961451247165533,
  0.7505668934240363,
  0.7097505668934241,
  0.7732426303854876,
  0.7687074829931972,
  0.7369614512471655],
 'test_score': 0.7945205479452054,
 'train_loss_history': [1.1509819352948987,
  0.945921003818512,
  0.8122080242311632,
  0.751708149104505,
  0.743634190108325,
  0.7226126612843694,
  0.6997873477033667,
  0.6083846905746976,
  0.6112446994394869,
  0.5666100415023597]}

### Second Stage (train on hand annotated datasets)

#### Build corpus

In [0]:
new_data_folder = '/content/drive/My Drive/Capstone/second_stage_data/'

In [50]:
gdp_data = pd.read_csv('/content/drive/My Drive/Capstone/second_stage_data/gdp_df_oversampled.csv')
gdp_data.head()

,Unnamed: 0,description,publishedAt,title,title_desc_sent_1,title_desc
0,50,"In New York, Dow Jones industrial average up 4...",2019-11-27,North American markets reach new record highs ...,1,North American markets reach new record highs ...
1,107,Canadian gross domestic product grew at the fa...,2018-12-21,Canada's GDP grows at fastest pace in 5 months...,1,Canada's GDP grows at fastest pace in 5 months...
2,123,U.S. President Donald Trump predicted data on ...,2018-07-27,Trump predicts data will show U.S. economy in ...,1,Trump predicts data will show U.S. economy in ...
3,80,Canadian firms among many investing billions i...,2019-07-17,Companies look to cash in on out-of-this-world...,1,Companies look to cash in on out-of-this-world...
4,61,That’s despite new revenues including 3% tax o...,2019-09-30,Liberals promises new spending and four more y...,1,Liberals promises new spending and four more y...


In [51]:
new_df = gdp_data[['title_desc_sent_1', 'title_desc']]
new_df.columns= ['label', 'text']
new_df.tail()

,label,text
160,0,"Vancouver needs 10,000 affordable housing unit..."
161,0,Lessons from the Great Crash of 1929 for inves...
162,0,Two more months of traffic pain until CN finis...
163,0,Fed signals patience on rate moves ‘for some t...
164,0,"With little to say on coronavirus, is Quebec's..."


In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split

import sklearn

In [0]:
kaggle_X_train, kaggle_X_te, kaggle_Y_train, kaggle_Y_te = train_test_split(new_df['text'],new_df['label'], test_size = 0.2, random_state = 0, stratify = new_df['label'])
kaggle_X_dev, kaggle_X_test, kaggle_Y_dev, kaggle_Y_test = train_test_split(kaggle_X_te,kaggle_Y_te, test_size = 0.5, random_state = 0)

In [0]:
benchmark_train_df =  pd.DataFrame({ 'label': kaggle_Y_train, 'text': kaggle_X_train   })
benchmark_dev_df = pd.DataFrame({ 'label': kaggle_Y_dev , 'text': kaggle_X_dev  })
benchmark_test_df = pd.DataFrame({ 'label': kaggle_Y_test, 'text': kaggle_X_test   })

In [0]:
benchmark_train_df.to_csv( new_data_folder + "train.csv",  sep='\t', index = False, header = False)
benchmark_dev_df.to_csv( new_data_folder + "dev.csv",sep='\t', index = False, header = False)
benchmark_test_df.to_csv( new_data_folder + "test.csv", sep='\t',index = False, header = False)

In [57]:
x = pd.read_csv('/content/drive/My Drive/Capstone/second_stage_data/train.csv', delimiter = '\t')
x.head()

,1,"U.S. economy grew at unrevised 4.2% pace in second quarter. The U.S. economy grew in the second quarter at an unrevised 4.2 per cent pace, the fastest since late 2014, indicating a solid foundation for this quarter, Commerce Department data showed Thursday."
0,-1,"Alberta government focuses on high debt, not i..."
1,0,Statistics are great unless they measure the w...
2,0,"Vancouver needs 10,000 affordable housing unit..."
3,-1,U.S. fourth-quarter GDP revisions show weaker ...
4,1,The Toronto Raptors' NBA playoff run helped bo...


In [58]:

new_column_name_map = {1: "text", 0: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(new_data_folder,
                                         new_column_name_map,
                                         skip_header=False, #no header in kaggle data
                                         delimiter='\t',    # comma separated rows
)

2020-05-22 01:12:34,392 Reading data from /content/drive/My Drive/Capstone/second_stage_data
2020-05-22 01:12:34,394 Train: /content/drive/My Drive/Capstone/second_stage_data/train.csv
2020-05-22 01:12:34,395 Dev: /content/drive/My Drive/Capstone/second_stage_data/dev.csv
2020-05-22 01:12:34,397 Test: /content/drive/My Drive/Capstone/second_stage_data/test.csv


#### Second Stage fine-tuning

In [59]:
# benchmark_embeddings = FlairEmbeddings(data_folder + 'best-model.pt')
benchmark_classifier = TextClassifier.load(data_folder + 'best-model.pt')

2020-05-22 01:12:39,079 loading file /content/drive/My Drive/Capstone/data/best-model.pt


In [60]:
trainer = ModelTrainer(benchmark_classifier, corpus)
trainer.train(new_data_folder, max_epochs=10)

2020-05-22 01:12:53,841 ----------------------------------------------------------------------------------------------------
2020-05-22 01:12:53,849 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_feat

{'dev_loss_history': [1.1998101472854614,
  1.0167970657348633,
  1.59540855884552,
  1.1989225149154663,
  1.1810652017593384,
  0.7167848348617554,
  1.9236620664596558,
  0.7530584931373596,
  1.4163832664489746,
  0.7204463481903076],
 'dev_score_history': [0.6666666666666666,
  0.625,
  0.6666666666666666,
  0.7083333333333334,
  0.7083333333333334,
  0.7083333333333334,
  0.625,
  0.75,
  0.75,
  0.7916666666666666],
 'test_score': 1.0,
 'train_loss_history': [1.1255870461463928,
  0.8426044344902038,
  0.7767580091953278,
  0.9558311581611634,
  0.7900098085403442,
  0.5427311539649964,
  0.3136239111423492,
  0.3569527447223663,
  0.440702486038208,
  0.35590792894363404]}

In [8]:
final_classifier = TextClassifier.load(new_data_folder + 'final-model.pt')

2020-05-22 02:17:46,944 loading file /content/drive/My Drive/Capstone/second_stage_data/final-model.pt


In [10]:
final_classifier.predict("Canada's economy shrinks in October from auto strike spillover. Canada’s economy contracted in October for the first time in eight months, as the United Auto Workers strike in the U.S. weighed on plant production")

[Sentence: "Canada's economy shrinks in October from auto strike spillover. Canada’s economy contracted in October for the first time in eight months, as the United Auto Workers strike in the U.S. weighed on plant production"   [− Tokens: 34  − Sentence-Labels: {'class': [-1 (0.9992)]}]]

In [11]:
final_classifier.predict("The candidates: Great Slave. Patrick Scott, Katrina Nokleby face off for seat vacated by former minister Glen Abernethy")

[Sentence: "The candidates: Great Slave. Patrick Scott, Katrina Nokleby face off for seat vacated by former minister Glen Abernethy"   [− Tokens: 18  − Sentence-Labels: {'class': [0 (0.9757)]}]]

In [12]:
final_classifier.predict("North American markets reach new record highs on stronger U.S. economic growth. In New York, Dow Jones industrial average up 42.32 points to 28,164")

[Sentence: "North American markets reach new record highs on stronger U.S. economic growth. In New York, Dow Jones industrial average up 42.32 points to 28,164"   [− Tokens: 24  − Sentence-Labels: {'class': [1 (0.9907)]}]]

In [9]:
final_classifier.predict("There's a lot of economic uncertainty right now as the world tries to curb the spread of COVID-19. But will the Bank of Canada's cut to interest rates help our economy? And how do events know when to pull the plug and cancel it all?")

[Sentence: "There's a lot of economic uncertainty right now as the world tries to curb the spread of COVID-19. But will the Bank of Canada's cut to interest rates help our economy? And how do events know when to pull the plug and cancel it all?"   [− Tokens: 45  − Sentence-Labels: {'class': [-1 (0.5661)]}]]